# FERNANDO LEON FRANCO

In [87]:
import numpy as np
import polars as pl
from bs4 import BeautifulSoup
import os
from colorstreak import log


# MEJORAR LA CARGA DESDE UN CSV

In [88]:
def generar_csv(path_lectura, path_guardado):
    # Obtener la ruta absoluta del directorio actual
    # Leerlo y convertirlo en un DataFrame .csv para manipularlo mejor
    with open(path_lectura, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        
    data = [line.strip().split(":::") for line in lines]


    df = pl.DataFrame(data, schema=["id", "genero", "pais"], orient="row")


    # Guardar el DataFrame como un archivo .csv
    df.write_csv(path_guardado)
    return True

# ==================== Configuración de rutas =====================


base_path = os.getcwd()

# ===================== Creación de CSV de datos de prueba =====================
ruta_lectura_prueba = os.path.join(base_path, "data/author_profiling/es_test/truth.txt")
ruta_guardado_prueba = os.path.join(base_path, "data/author_profiling/es_test/truth_test.csv")

creado = generar_csv(ruta_lectura_prueba, ruta_guardado_prueba)

if creado:
    log.info(f'Archivo CSV creado en: "{ruta_guardado_prueba}"')
# ==============================================================================


# ===================== Creación de CSV de datos de entrenamiento ==============
ruta_lectura_entrenamiento = os.path.join(base_path, "data/author_profiling/es_train/truth.txt")
ruta_guardado_entrenamiento = os.path.join(base_path, "data/author_profiling/es_train/truth_train.csv")

creado_entrenamiento = generar_csv(ruta_lectura_entrenamiento, ruta_guardado_entrenamiento)
if creado_entrenamiento:
    log.info(f'Archivo CSV creado en: "{ruta_guardado_entrenamiento}"')
# ==============================================================================



[INFO] Archivo CSV creado en: "/Users/ferleon/Github/semestre_v/procesamiento_lenguaje/data/author_profiling/es_test/truth_test.csv"
[INFO] Archivo CSV creado en: "/Users/ferleon/Github/semestre_v/procesamiento_lenguaje/data/author_profiling/es_train/truth_train.csv"


# CARGAR LOS INDICES DESDE TRUTH DESDE EL CSV

In [89]:
def cargar_xml(id_archivo, train=True):
    ruta_base = os.path.join(base_path, "data/author_profiling/es_test" if not train else "data/author_profiling/es_train")
    ruta_archivo = os.path.join(ruta_base, f"{id_archivo}.xml")
    log.info(f"ruta archivo: {ruta_archivo}")
    
    with open(ruta_archivo, 'r', encoding='utf-8') as file:
        contenido = file.read()
        
    log.debug(f'Contenido del archivo: {contenido[:1000]}...')  

# ===================== Cargar CSV de datos de entrenamiento ==================
 
df_train = pl.read_csv(ruta_guardado_entrenamiento)
df_train.head()

id_archivo = df_train['id'][0]
log.debug(f'ID del archivo: {id_archivo}')

cargar_xml(id_archivo)


[DEBUG] ID del archivo: 74bcc9b0882c8440716ff370494aea09
[INFO] ruta archivo: /Users/ferleon/Github/semestre_v/procesamiento_lenguaje/data/author_profiling/es_train/74bcc9b0882c8440716ff370494aea09.xml
[DEBUG] Contenido del archivo: <author lang="es">
	<documents>
		<document><![CDATA[Tiene que valer la pena que esté despierta a esta hora #HoylosGrammycon40]]></document>
		<document><![CDATA[Tintas chinas, si ven ésto, estoy buscándolas por todas partes, aparezcan que mañana hay plásticas, ¿sí? Xoxo]]></document>
		<document><![CDATA["Maestro no le abrió!" -Ay qué pesar contigo Yuri Yanet.]]></document>
		<document><![CDATA[Lo bueno de no estar enamorado es que nadie se fastidia con los 53,2197 tweets depresivos corta venas que ecriben al día.]]></document>
		<document><![CDATA[Recordaré este día como el día que duró 2568 semanas.]]></document>
		<document><![CDATA[Que vuelvas y te quedes para siempre.]]></document>
		<document><![CDATA[Resulta que el punto al lado de la luna de hoy er